In [1]:
import json
import dask
import dask_awkward as dak
import awkward as ak
import matplotlib.pyplot
from coffea import dataset_tools
from coffea.nanoevents import NanoEventsFactory, PFNanoAODSchema
import pickle
import os
import warnings

In [2]:
warnings.filterwarnings("ignore", module="coffea.*")
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")
warnings.filterwarnings("ignore", "invalid value")

In [10]:
samples_path = '/project01/ndcms/cmoore24/samples'
filelist = {}
categories = os.listdir(samples_path)
for i in categories:
    if '.root' in os.listdir(f'{samples_path}/{i}')[0]:
        files = os.listdir(f'{samples_path}/{i}')
        filelist[i] = [f'{samples_path}/{i}/{file}' for file in files]
    else:
        sub_cats = os.listdir(f'{samples_path}/{i}')
        for j in sub_cats:
            if '.root' in os.listdir(f'{samples_path}/{i}/{j}')[0]:
                files = os.listdir(f'{samples_path}/{i}/{j}')
                filelist[f'{i}_{j}'] = [f'{samples_path}/{i}/{j}/{file}' for file in files]

In [14]:
dict = {}
for i in filelist:
    dict[i] = {}
    dict[i]['files'] = {}
    for j in filelist[i]:
        dict[i]['files'][j] = {'object_path': 'Events'}

In [17]:
with open('input_datasets.json', 'w') as fin:
    json.dump(dict, fin)

In [26]:
@dask.delayed
def sampler(samples):
    samples_ready, samples = dataset_tools.preprocess(
        samples,
        step_size=50_000,
        skip_bad_files=True,
        recalculate_steps=True,
        save_form=False,
    )
    return samples_ready

In [27]:
sampler_dict = {}
for i in dict:
    sampler_dict[i] = sampler(dict[i])

In [36]:
with open('samples_ready.json', 'r') as f:
    b = json.load(f)